In [2]:
# Imports from sympt
from sympt import *
# imports from sympy
import sympy as sp

# Setup

In [3]:
# ---------------- Defining the symbols ------------------
# Order 0
omega0 = RDSymbol('omega0', real=True, positive=True)
theta = RDSymbol(r'\theta', real=True)
n = sp.Matrix([0, sp.cos(theta), sp.sin(theta)])

# Order 1
omega = RDSymbol('omega', real=True, positive=True, order=1)
Eac = RDSymbol(r'\tilde{E}_{\mathrm{ac}}', real=True, order=1)
Ez = RDSymbol(r'E_{z}', real=True, order=1)
bsl = RDSymbol(r'\tilde{b}_{\mathrm{SL}}', real=True, order=1)


# ----------------- Defining the basis -------------------
# Spin basis: Finite 2x2 Hilbert space
Spin = RDBasis('sigma', 2)
s0, sx, sy, sz = Spin.basis # Pauli operators
s_vec = sp.Matrix([sx, sy, sz]) # Spin vector
# Boson basis: Infinite bosonic Hilbert space
a = BosonOp('a')
ad = Dagger(a)

# -------------- Defining the Hamiltonian ----------------
# Unperturbed Hamiltonian H0
H0 = hbar * omega0 * (ad*a + sp.Rational(1,2))
# Perturbation Hamiltonians
HE = Eac * sp.sin(omega * t) * (ad + a)
HB = -sp.Rational(1,2) * Ez * sz
V = bsl * (ad + a) * n.dot(s_vec)

display_dict({
    sp.Symbol('H_0'): H0,
    sp.Symbol('H_E'): HE,
    sp.Symbol('H_B'): HB,
    sp.Symbol(r'V'): V,
})

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [4]:
# Deffining Effective Hamiltonian Object
Eff_Frame = EffectiveFrame(H=H0 + HB, V=HE + V, subspaces=[Spin])

The EffectiveFrame object has been initialized successfully.


# Time Dependent Schrieffer-Wolff Transformation

In [5]:
# Calculate the effective model using the Schrieffer-Wolff transformation up to the second order
Eff_Frame.solve(max_order=5, method="SW")
# Obtaining the result in the matrix form
H_eff = Eff_Frame.get_H(return_form='matrix')

Performing Time Dependent SWT for each order: 100%|█████████████████████████| 5/5 [00:22<00:00,  4.59s/it]
The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.
Converting to matrix form: 100%|████████████████████████████████████████████| 6/6 [00:00<00:00, 33.98it/s]


In [6]:
# Corrections to the Hamiltonian for each order
H0, H1, H2, H3, H4, H5 = [H_correction for H_correction in Eff_Frame.corrections.values()]

# Simplifying corrections

In [7]:
# Expectation Values for the bosonic operators
# <n=0|a|n=0> = 0
# <n=0|ad|n=0> = 0
# <n=0|ad*a|n=0> = 0
subs_expvals = {
    ad*a: 0,
    a : 0,
    ad : 0,
}

In [9]:
# Projecting first order correction in spin subspace
H1_op = Spin.project(H1.expand())
H1_op

E_{z}*(sigma_0/2 - sigma_3/2)/2 - E_{z}*(sigma_0/2 + sigma_3/2)/2

In [15]:
# Projecting second order correction in spin subspace
H2_op = Spin.project(sp.expand_complex(H2.expand()).trigsimp())
H2_op

-2*I*\tilde{E}_{\mathrm{ac}}*\tilde{b}_{\mathrm{SL}}*sin(omega*t)*cos(\theta)*(sigma_1/2 - I*sigma_2/2)/(hbar*omega0) + 2*I*\tilde{E}_{\mathrm{ac}}*\tilde{b}_{\mathrm{SL}}*sin(omega*t)*cos(\theta)*(sigma_1/2 + I*sigma_2/2)/(hbar*omega0) + (-\tilde{E}_{\mathrm{ac}}**2*sin(omega*t)**2 + 2*\tilde{E}_{\mathrm{ac}}*\tilde{b}_{\mathrm{SL}}*sin(\theta)*sin(omega*t) - \tilde{b}_{\mathrm{SL}}**2)*(sigma_0/2 - sigma_3/2)/(hbar*omega0) - (\tilde{E}_{\mathrm{ac}}**2*sin(omega*t)**2 + 2*\tilde{E}_{\mathrm{ac}}*\tilde{b}_{\mathrm{SL}}*sin(\theta)*sin(omega*t) + \tilde{b}_{\mathrm{SL}}**2)*(sigma_0/2 + sigma_3/2)/(hbar*omega0)

In [16]:
# Projecting third order correction in spin subspace
H3_op = Spin.project(H3.expand().subs(subs_expvals))
H3_op

-I*E_{z}*\tilde{b}_{\mathrm{SL}}**2*sin(\theta)*cos(\theta)*(sigma_1/2 - I*sigma_2/2)/(hbar**2*omega0**2) + I*E_{z}*\tilde{b}_{\mathrm{SL}}**2*sin(\theta)*cos(\theta)*(sigma_1/2 + I*sigma_2/2)/(hbar**2*omega0**2) - E_{z}*\tilde{b}_{\mathrm{SL}}**2*cos(\theta)**2*(sigma_0/2 - sigma_3/2)/(hbar**2*omega0**2) + E_{z}*\tilde{b}_{\mathrm{SL}}**2*cos(\theta)**2*(sigma_0/2 + sigma_3/2)/(hbar**2*omega0**2)

In [17]:
# Projecting fourth order correction in spin subspace
H4_op = Spin.project(sp.expand_complex(H4.xreplace(subs_expvals).expand())).collect(-2*bsl*Eac*sp.cos(theta)*sp.sin(omega*t))
H4_op

I*(-E_{z}**2*\tilde{E}_{\mathrm{ac}}*\tilde{b}_{\mathrm{SL}}*sin(omega*t)*cos(\theta)/(hbar**3*omega0**3) - \tilde{E}_{\mathrm{ac}}*\tilde{b}_{\mathrm{SL}}*omega**2*sin(omega*t)*cos(\theta)/(hbar*omega0**3))*(sigma_1/2 - I*sigma_2/2) + I*(E_{z}**2*\tilde{E}_{\mathrm{ac}}*\tilde{b}_{\mathrm{SL}}*sin(omega*t)*cos(\theta)/(hbar**3*omega0**3) + \tilde{E}_{\mathrm{ac}}*\tilde{b}_{\mathrm{SL}}*omega**2*sin(omega*t)*cos(\theta)/(hbar*omega0**3))*(sigma_1/2 + I*sigma_2/2) + (-E_{z}**2*\tilde{b}_{\mathrm{SL}}**2*cos(\theta)**2/(hbar**3*omega0**3) + \tilde{E}_{\mathrm{ac}}**2*omega**2*cos(2*omega*t)/(2*hbar*omega0**3) - \tilde{E}_{\mathrm{ac}}**2*omega**2/(2*hbar*omega0**3) - \tilde{E}_{\mathrm{ac}}*\tilde{b}_{\mathrm{SL}}*omega**2*sin(\theta)*sin(omega*t)/(hbar*omega0**3))*(sigma_0/2 + sigma_3/2) + (-E_{z}**2*\tilde{b}_{\mathrm{SL}}**2*cos(\theta)**2/(hbar**3*omega0**3) + \tilde{E}_{\mathrm{ac}}**2*omega**2*cos(2*omega*t)/(2*hbar*omega0**3) - \tilde{E}_{\mathrm{ac}}**2*omega**2/(2*hbar*omega0**

In [18]:
# Projecting fifth order correction in spin subspace
H5_op = Spin.project(sp.expand_complex(H5.xreplace(subs_expvals).expand())).trigsimp()
H5_op

I*E_{z}*\tilde{b}_{\mathrm{SL}}**2*(-E_{z}**2 - 4*\tilde{E}_{\mathrm{ac}}**2*sin(omega*t)**2 + \tilde{b}_{\mathrm{SL}}**2)*sin(\theta)*cos(\theta)*(sigma_1 - I*sigma_2)/(2*hbar**4*omega0**4) + E_{z}*\tilde{b}_{\mathrm{SL}}**2*(-E_{z}**2 - 4*\tilde{E}_{\mathrm{ac}}**2*sin(omega*t)**2 + \tilde{b}_{\mathrm{SL}}**2)*cos(\theta)**2*(sigma_0 - sigma_3)/(2*hbar**4*omega0**4) + I*E_{z}*\tilde{b}_{\mathrm{SL}}**2*(E_{z}**2 + 4*\tilde{E}_{\mathrm{ac}}**2*sin(omega*t)**2 - \tilde{b}_{\mathrm{SL}}**2)*sin(\theta)*cos(\theta)*(sigma_1 + I*sigma_2)/(2*hbar**4*omega0**4) + E_{z}*\tilde{b}_{\mathrm{SL}}**2*(E_{z}**2 + 4*\tilde{E}_{\mathrm{ac}}**2*sin(omega*t)**2 - \tilde{b}_{\mathrm{SL}}**2)*cos(\theta)**2*(sigma_0 + sigma_3)/(2*hbar**4*omega0**4)